# Tool recommendation with Attention network 
## (Gated recurrent units Attention neural network with weighted cross-entropy loss)

In [84]:
import numpy as np
import json
import warnings
import operator

import tensorflow as tf
from keras import backend as K
import h5py

warnings.filterwarnings("ignore")


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        self.units = units
        super(BahdanauAttention, self).__init__(**kwargs)

    def get_config(self):
        return super().get_config().copy()

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class ToolPredictionAttentionModel():
  
    def __init__(self, parameters):
        self.embedding_size = int(parameters["embedding_size"])
        self.gru_units = int(parameters["units"])
        self.max_len = parameters["max_len"]
        self.dimensions = parameters["dimensions"]
        self.learning_rate = parameters["learning_rate"]
        self.class_weights = parameters["class_weights"]
        self.spatial_dropout = parameters["spatial_dropout"]
        self.recurrent_dropout = parameters["recurrent_dropout"]
        self.dropout = parameters["dropout"]
        
    def weighted_loss(self, class_weights):
        """
        Create a weighted loss function. Penalise the misclassification
        of classes more with the higher usage
        """
        weight_values = list(class_weights.values())

        def weighted_binary_crossentropy(y_true, y_pred):
            # add another dimension to compute dot product
            expanded_weights = tf.keras.backend.expand_dims(weight_values, axis=-1)
            return tf.keras.backend.dot(tf.keras.backend.binary_crossentropy(y_true, y_pred), expanded_weights)
        return weighted_binary_crossentropy

    def create_model(self):
        sequence_input = tf.keras.layers.Input(shape=(self.max_len,), dtype='int32')
        embedded_sequences = tf.keras.layers.Embedding(self.dimensions, self.embedding_size, input_length=self.max_len, mask_zero=True)(sequence_input)
        embedded_sequences_dropout = tf.keras.layers.SpatialDropout1D(self.spatial_dropout)(embedded_sequences)
        gru = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=True,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )
        sample_output, sample_hidden = gru(embedded_sequences_dropout, initial_state=None)
        attention = BahdanauAttention(self.gru_units)
        context_vector, attention_weights = attention(sample_hidden, sample_output)
        dropout = tf.keras.layers.Dropout(self.dropout)(context_vector)
        output = tf.keras.layers.Dense(self.dimensions, activation='sigmoid')(dropout)
        model = tf.keras.Model(inputs=sequence_input, outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate),
            loss=self.weighted_loss(self.class_weights),
        )
        return model


def load_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    best_parameters = json.loads(trained_model.get('parameters').value)
    compatible_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    new_model = ToolPredictionAttentionModel(best_parameters).create_model()
    new_model.load_weights(model_path)
    return dictionary, reverse_dictionary, class_weights, best_parameters, compatible_tools, new_model


model_path = "data/tool_recommendation_attention_model.hdf5"
dictionary, reverse_dictionary, class_weights, best_parameters, compatible_tools, new_model = load_model(model_path)

## Unpack trained model for prediction

In [85]:
print(reverse_dictionary)

{'1': 'sam_bw_filter', '2': 'gatk2_print_reads', '3': 'bismark_bowtie2', '4': 'ctb_simsearch', '5': 'proteomics_search_tandem_1', '6': 'cshl_awk_replace_in_column', '7': 'tophat2', '8': 'seq_filter_by_mapping', '9': 'aragorn_trna', '10': 'cufflinks', '11': 'EMBOSS: geecee41', '12': 'FalseDiscoveryRate', '13': 'vt_normalize', '14': 'ctb_compound_convert', '15': 'picard_ARRG', '16': 'extract_bcs.py', '17': 'bedtools_bamtofastq', '18': 'bwa_mem', '19': 'cuffmerge', '20': 'cshl_awk_tool', '21': 'subtract_query1', '22': 'picard_MarkDuplicates', '23': 'cshl_multijoin', '24': 'term_id_vs_term_name', '25': 'bowtie2', '26': 'methtools_plot', '27': 'picard_CASM', '28': 'ctb_filter', '29': 'MzTabExporter', '30': 'bcftools_view', '31': 'Remove_ending', '32': 'cshl_fastx_quality_statistics', '33': 'cshl_fastq_quality_boxplot', '34': 'ncbi_blastn_wrapper', '35': 'methtools_calling', '36': 'rmcontamination', '37': 'gatk2_base_recalibrator', '38': 'infernal_cmsearch', '39': 'bedtools_bedtobam', '40': 

In [86]:
def compute_recommendations(tool_sequence, labels, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = new_model.predict(sample_reshaped, verbose=0)
    class_weighs = best_parameters["class_weights"]
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    #prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_pos = [item for item in topk_prediction_pos if item != 0]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

## Indices of tools

In [87]:
topk = 20 # set the maximum number of recommendations
tool_seq = "2" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(tool_seq, "", topk)

Current tool sequence: 

gatk2_print_reads

Recommended tools for the tool sequence 'gatk2_print_reads' with their scores in decreasing order:

gatk2_reduce_reads(100%)
r_correlation_matrix(99%)
Show tail1(99%)
Remove_ending(97%)
EMBOSS: fuzztran39(97%)
msconvert3_raw(97%)
eukaryotic_ncbi_submission(96%)
modencode_peakcalling_macs2(96%)
openms_protein_quantifier(95%)
bam2wig(95%)
heatmapper(95%)
prokaryotic_ncbi_submission(95%)
cshl_multijoin(95%)
htseq-count(95%)
heatmapper_deepTools(94%)
blockbuster(94%)
silac_analyzer(93%)
deeptools_correctGCBias(92%)
cshl_sed_tool(92%)

Tool ids:

gatk2_reduce_reads(107)
r_correlation_matrix(303)
Show tail1(243)
Remove_ending(31)
EMBOSS: fuzztran39(193)
msconvert3_raw(251)
eukaryotic_ncbi_submission(73)
modencode_peakcalling_macs2(191)
openms_protein_quantifier(129)
bam2wig(205)
heatmapper(280)
prokaryotic_ncbi_submission(60)
cshl_multijoin(23)
htseq-count(113)
heatmapper_deepTools(56)
blockbuster(255)
silac_analyzer(297)
deeptools_correctGCBias(41

## Recommended tools

In [88]:
print(new_model.summary())

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 25)]              0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 25, 33)            10230     
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 25, 33)            0         
_________________________________________________________________
gru_18 (GRU)                 [(None, 25, 85), (None, 8 30600     
_________________________________________________________________
bahdanau_attention_15 (Bahda ((None, 85), (None, 25, 1 14706     
_________________________________________________________________
dropout_4 (Dropout)          (None, 85)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 310)               266

In [89]:
print(best_parameters)

{'batch_size': 51.0, 'dropout': 0.22602806806492431, 'embedding_size': 33.0, 'learning_rate': 0.0032540638980664565, 'recurrent_dropout': 0.4216428563134349, 'spatial_dropout': 0.07267624883456103, 'units': 85.0, 'max_len': 25, 'dimensions': 310, 'class_weights': {'0': 0.0, '1': 1.2679033691330028, '2': 0.0953052070646881, '3': 2.36302973908641, '4': 0.3291822433584368, '5': 0.5994148283307562, '6': 0.0, '7': 4.893910221874146, '8': 0.38600688549788165, '9': 1.9296225475826647, '10': 5.249888185668796, '11': 1.2584609896100056, '12': 5.5060777871737265, '13': 0.2527023535557542, '14': 0.09531017980432493, '15': 0.09531017980432493, '16': 0.38668123051024755, '17': 4.116595171156921, '18': 7.360040357734192, '19': 3.6149637711637683, '20': 0.09531017980432493, '21': 0.1879510232386989, '22': 5.78484241624876, '23': 0.0, '24': 0.09543005443725577, '25': 7.13966033596492, '26': 0.09473615144520271, '27': 2.7111174575911785, '28': 0.0, '29': 2.1856234256723175, '30': 0.02803959057123678, '